# Load CSV file to SQL structure

In this file, we perform the proccess of reading and loading CSV file obtained for a candidate's job test using Python Kernel in a venv.

In this process, the following will be carried out:

1. Connection with the **etl_workshop_first** database hosted in a **Local server** or in **AWS RDS server** depending on the credendials filename assignment (_credentials.ini_ filename for local or *aws_credentials.ini* filename for cloud). If the database doesn't exists, it will be created.

2. Creation of the database's structure, **raw_applicant** and **applicant** tables. As the proccess is running, the tables will be reset if they already exists.

3. Reading of CSV file.

4. Loading of data to **raw_applicant** table in **etl_workshop_first** database.

---
## Install requirement libraries

In [ ]:
pip install -r ../code/config/requirements.txt

---
## Proccess

Import libraries:

In [2]:
import pandas as pd
import importlib.util

spec = importlib.util.spec_from_file_location("connect_database", f"../code/connect_database.py")
connect_database = importlib.util.module_from_spec(spec)
spec.loader.exec_module(connect_database)

Information about CSV file to load:

In [3]:
location_file = '../data/candidates.csv'
raw_table_database = 'raw_applicant'
names_file = ['first_name', 'last_name', 'email', 'applicant_date', 'country', 'experience_year', 'seniority', 'technology', 'code_challenge_score', 'technical_interview_score']

Create connection:

In [4]:
connection = connect_database.ConnectionPostgres()
connection.make_tables()

Let's read CSV file and create the dataframe

In [5]:
raw_data_dataframe = pd.read_csv(location_file, names = names_file, delimiter=';', header=0)
raw_data_dataframe.sort_values(by=['first_name', 'last_name', 'applicant_date', 'country'], inplace=True)
raw_data_dataframe.index += 1
raw_data_dataframe.reset_index(inplace=True)
raw_data_dataframe.rename(columns={'index': 'id'}, inplace=True)

Send data for PostgreSQL:

In [6]:
raw_data_dataframe.to_sql(raw_table_database, connection.engine, if_exists='replace', index=False)
connection.log('Data loaded to {}: {} rows - {} columns.' .format(connection.connection_config['database'], raw_data_dataframe.shape[0], raw_data_dataframe.shape[1]))

connection.close_connection()

Data loaded:

![Raw applicant data](https://gist.github.com/assets/92474551/35a5912d-65dd-4680-aca3-d6b7e0b240f3)